In [ ]:
#pragma cling add_include_path("$HOME/miniconda3/envs/cling-torch/lib/python3.7/site-packages/torch/include")
#pragma cling add_include_path("$HOME/miniconda3/envs/cling-torch/lib/python3.7/site-packages/torch/include/torch/csrc/api/include")
#pragma cling add_library_path("$HOME/miniconda3/envs/cling-torch/lib/python3.7/site-packages/torch/lib")
#pragma cling load("libtorch")

In [ ]:
#include <ATen/ATen.h>

at::Tensor a = at::ones({2, 2}, at::kInt);
at::Tensor b = at::randn({2, 2});
auto c = a + b.to(at::kInt);

std::cout << "a: " << a << std::endl;
std::cout << std::endl;
std::cout << "b: " << b << std::endl;
std::cout << std::endl;
std::cout << "c: " << c << std::endl;

In [ ]:
#include <torch/torch.h>
#include <torch/csrc/autograd/variable.h>
#include <torch/csrc/autograd/function.h>

torch::Tensor a_tensor = torch::ones({2, 2}, torch::requires_grad());
torch::Tensor b_tensor = torch::randn({2, 2});

std::cout << a_tensor << std::endl;
std::cout << b_tensor << std::endl;

auto c_tensor = a_tensor + b_tensor;
c_tensor.backward(); // a.grad() will now hold the gradient of c w.r.t a

std::cout << c_tensor << std::endl;